In [ ]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.svm import SVR

In [ ]:
df = pd.read_csv(r'laptop_price.csv')

In [ ]:
print(df)

In [ ]:
df.head()
df.info()
df.shape
df.duplicated().sum()       # To find how many duplicate values are there
df.isnull().sum()           # To find how many null values are there

In [ ]:
df.drop(columns=['Unnamed: 13'],inplace=True)   #Drop Unnamed Columns
df.head()
df['Ram'] = df['Ram'].astype('str').str.replace('GB','')    #To remove GB from Ram and convert into to number from string
df['Weight'] = df['Weight'].astype('str').str.replace('kg','')    # To remove the kg from Weight and convert it into number from string
df['Ram']=df['Ram'].astype('int32')
df['Weight']=df['Weight'].astype('float32')
df.head()
df.info()

In [ ]:
sns.distplot(df['Price_euros'])

#Observation will be data is skewed.

In [ ]:
df['Company'].value_counts().plot(kind='bar')

In [ ]:
sns.barplot(x=df['Company'],y=df['Price_euros'])
plt.xticks(rotation='vertical')


In [ ]:
df['TypeName'].value_counts().plot(kind='bar')

#Observation : Notebooks are highly priced

In [ ]:
sns.barplot(x=df['TypeName'],y=df['Price_euros'])
plt.xticks(rotation='vertical')

#Onservation : Workstation > Gaming > ...

In [ ]:
sns.scatterplot(x=df['Inches'],y=df['Price_euros'])

#Observation : As size increases, prices are increasing.

In [ ]:
df['ScreenResolution'].value_counts()


#Feature engineering : Find laptop is touch screen or not

In [ ]:
df['TouchScreen'] = df['ScreenResolution'].apply(lambda x:1 if 'Touchscreen' in x else 0)
df.info()
df.sample()

df['TouchScreen'].value_counts().plot(kind='bar')


In [ ]:
sns.barplot(x=df['TouchScreen'],y=df['Price_euros'])

################SHOWS THAT TOUCHSCREEN IS IMP!!!!!!!!!

In [ ]:
df['IPS'] = df['ScreenResolution'].apply(lambda x:1 if 'IPS' in x else 0)
df['IPS'].value_counts().plot(kind='bar')
# sns.barplot(x=df['IPS'],y=df['Price_euros'])
######### IPS panel is there or not is imp!!!!!!!!

In [ ]:
newdf = df['ScreenResolution'].str.split('x',n=1,expand=True)
df['X_Resolution'] = newdf[0]   #Still problematic
df['X_Resolution'] = df['X_Resolution'].str.replace(',','').str.findall(r'(\d+\.?\d+)').apply(lambda x : x[-1])
df['Y_Resolution'] = newdf[1]
df['X_Resolution'] = df['X_Resolution'].astype('int32')
df['Y_Resolution'] = df['Y_Resolution'].astype('int32')

df.head()

# df.sample()

In [ ]:
df.info()
df.corr()['Price_euros']

In [ ]:
df['PPI'] = ((((df['X_Resolution']**2) + (df['Y_Resolution']**2))**0.5)/df['Inches']).astype('float')

In [ ]:
df.drop(columns=['ScreenResolution'],inplace=True)

In [ ]:
df.info()
df['Cpu'].value_counts().plot(kind='bar')
# plt.xticks(rotation='vertical')

In [ ]:
df['Cpu Name'] = df['Cpu'].apply(lambda x : " ".join(x.split()[0:3]))
df['Cpu Name']

In [ ]:
def fetch_processor_name(text) : 
    if text=='Intel Core i7' or text == 'Intel Core i5' or text == 'Intel Core i3':
        return text
    else:
        if text.split()[0] == 'Intel':
            return 'Other Intel Processor'
        else:
            return 'AMD Processor'


In [ ]:
        
df['CPU Brand'] = df['Cpu Name'].apply(fetch_processor_name)
df.head()

In [ ]:
df['CPU Brand'].value_counts().plot(kind='bar')

In [ ]:
df['Gpu_brand'] = df['Gpu'].apply(lambda x:x.split()[0])
df = df[df['Gpu_brand'] != 'ARM']
df.drop(columns=['Gpu'],inplace=True)

df.head()

In [ ]:
def getOS(input):
    if input == 'Windows 10' or input == 'Windows 7' or input == 'Windows 10 S':
        return 'Windows'
    elif input == 'macOS' or input == 'Mac OS X':
        return 'Mac'
    else:
        return 'Others'
df['os'] = df['OpSys'].apply(getOS)
df.drop(columns=['OpSys'],inplace=True)

df.head()

In [ ]:
# df['Memory'] = df['Memory'].str.replace('\.0', '', regex=True)
# df["Memory"] = df["Memory"].str.replace('GB', '')
# df["Memory"] = df["Memory"].str.replace('TB', '000')
# new = df["Memory"].str.split("+", n = 1, expand = True)
# df["first"]= new[0]
# df["first"]=df["first"].str.strip()
# df["second"]= new[1]
# df["Layer1HDD"] = df["first"].apply(lambda x: 1 if "HDD" in x else 0)
# df["Layer1SSD"] = df["first"].apply(lambda x: 1 if "SSD" in x else 0)
# df["Layer1Hybrid"] = df["first"].apply(lambda x: 1 if "Hybrid" in x else 0)
# df["Layer1Flash_Storage"] = df["first"].apply(lambda x: 1 if "Flash Storage" in x else 0)
# df['first'] = df['first'].str.replace(r'/D', '')
# df["second"].fillna("0", inplace = True)
# df["Layer2HDD"] = df["second"].apply(lambda x: 1 if "HDD" in x else 0)
# df["Layer2SSD"] = df["second"].apply(lambda x: 1 if "SSD" in x else 0)
# df["Layer2Hybrid"] = df["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
# df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
# df['second'] = df['second'].str.replace(r'D', '')

# df["Layer2HDD"] = df["second"].apply(lambda x: 1 if "HDD" in x else 0)
# df["Layer2SSD"] = df["second"].apply(lambda x: 1 if "SSD" in x else 0)
# df["Layer2Hybrid"] = df["second"].apply(lambda x: 1 if "Hybrid" in x else 0)
# df["Layer2Flash_Storage"] = df["second"].apply(lambda x: 1 if "Flash Storage" in x else 0)
# df['second'] = df['second'].str.replace(r'D', '')

# print(df["first"].remove())
# df["first"] = df["first"].astype(int)
# df["second"] = df["second"].astype(int)

# df["HDD"]=(df["first"]*df["Layer1HDD"]+df["second"]*df["Layer2HDD"])
# df["SSD"]=(df["first"]*df["Layer1SSD"]+df["second"]*df["Layer2SSD"])
# df["Hybrid"]=(df["first"]*df["Layer1Hybrid"]+df["second"]*df["Layer2Hybrid"])
# df["Flash_Storage"]=(df["first"]*df["Layer1Flash_Storage"]+df["second"]*df["Layer2Flash_Storage"])

# df.drop(columns=['first', 'second', 'Layer1HDD', 'Layer1SSD', 'Layer1Hybrid',
#        'Layer1Flash_Storage', 'Layer2HDD', 'Layer2SSD', 'Layer2Hybrid',
#        'Layer2Flash_Storage'],inplace=True)
df.drop(columns=['Memory'],inplace = True)

In [ ]:
x = df.drop(columns=['Price_euros'])

y = np.log(df['Price_euros'])

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=2)

In [ ]:
step1 = ColumnTransformer(transformers=[

('col_tnf',OneHotEncoder(sparse=False,drop='first'),[0,1,7,10,11])

],remainder='passthrough')

step2 = RandomForestRegressor(n_estimators=100,

random_state=3,

max_samples=0.5,

max_features=0.75,

max_depth=15)

pipe = Pipeline([

('step1',step1),

('step2',step2)

])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2 score',r2_score(y_test,y_pred))

print('MAE',mean_absolute_error(y_test,y_pred))